In [186]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [187]:
training = datasets.MNIST("data", train=True,  download=True, transform=ToTensor())
test = datasets.MNIST("data", train=False,  download=True, transform=ToTensor())

1.0%

100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100.0%
100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [188]:
train_dataloader = DataLoader(training, batch_size=64)
test_dataloader = DataLoader(test, batch_size=64)

In [189]:
cnn_train_dataloader = DataLoader(training, batch_size=64)
cnn_test_dataloader = DataLoader(test, batch_size=64)

In [190]:
for images, label in cnn_train_dataloader:
    images = images.view(-1, 1, 28, 28)
for images, label in cnn_test_dataloader:
    images = images.view(-1, 1, 28, 28)

In [191]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [192]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 300, device=device),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [193]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv_net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_net = nn.Sequential(
            nn.Linear(7*7*64, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = self.conv_net(x)
        x = x.view(x.size(0), -1)
        x = self.fc_net(x)
        return x

In [194]:
model = NeuralNet()
model.load_state_dict(torch.load("MNIST_weights.pth"))

<All keys matched successfully>

In [195]:
model_cnn = CNN()
model_cnn.load_state_dict(torch.load("MNIST_cnn_weights.pth"))

<All keys matched successfully>

In [196]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer_adam = torch.optim.Adam(model.parameters(), lr=1.8e-3)

In [213]:
loss_fn_cnn = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=2.52e-3)

In [198]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()

    for batch, (X, Y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, Y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"Loss: {loss.item()}, batch: {batch}")

In [199]:
def test_loop(dataloader, model, loss_fn, iteration):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for (X, Y) in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, Y)
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test {iteration+1} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [200]:
# epochs = 2
# for t in range(epochs):
#     train_loop(train_dataloader, model, loss_fn, optimizer_adam)
#     test_loop(test_dataloader, model, loss_fn, t)
# torch.save(model.state_dict(), "MNIST_weights.pth")

Loss: 2.5611479941289872e-05, batch: 0
Loss: 0.0004736580594908446, batch: 100
Loss: 0.00038856820901855826, batch: 200
Loss: 0.00012981124746147543, batch: 300
Loss: 0.003494004253298044, batch: 400
Loss: 0.011710887774825096, batch: 500
Loss: 0.00791223719716072, batch: 600
Loss: 0.00012096855789422989, batch: 700
Loss: 0.00045427068835124373, batch: 800
Loss: 0.0005726410308852792, batch: 900
Test 1 Error: 
 Accuracy: 97.1%, Avg loss: 0.192404 

Loss: 0.0001256070245290175, batch: 0
Loss: 0.0010707556502893567, batch: 100
Loss: 0.02220996469259262, batch: 200
Loss: 6.374660006258637e-05, batch: 300
Loss: 3.0176544896676205e-05, batch: 400
Loss: 0.0015852610813453794, batch: 500
Loss: 9.841487553785555e-06, batch: 600
Loss: 0.01866433583199978, batch: 700
Loss: 0.0006178961484692991, batch: 800
Loss: 0.0001836982264649123, batch: 900
Test 2 Error: 
 Accuracy: 97.8%, Avg loss: 0.146452 



In [214]:
epochs2 = 3
for t in range(epochs2):
    train_loop(cnn_train_dataloader, model_cnn, loss_fn_cnn, optimizer_cnn)
    test_loop(cnn_test_dataloader, model_cnn, loss_fn_cnn, t)
print("DONE")
torch.save(model_cnn.state_dict(), "MNIST_cnn_weights.pth")

Loss: 0.005144428927451372, batch: 0
Loss: 3.269657463533804e-05, batch: 100
Loss: 0.0009662464144639671, batch: 200
Loss: 0.0006048443028703332, batch: 300
Loss: 0.06644224375486374, batch: 400
Loss: 0.0011701879557222128, batch: 500
Loss: 0.001153750461526215, batch: 600
Loss: 0.031159499660134315, batch: 700
Loss: 0.012692232616245747, batch: 800
Loss: 0.010461484082043171, batch: 900
Test 1 Error: 
 Accuracy: 98.7%, Avg loss: 0.074482 

Loss: 0.042608119547367096, batch: 0
Loss: 0.0007200156687758863, batch: 100
Loss: 0.005838931538164616, batch: 200
Loss: 0.04209429398179054, batch: 300
Loss: 0.0006708378787152469, batch: 400
Loss: 0.0003395329404156655, batch: 500
Loss: 0.0026480327360332012, batch: 600
Loss: 0.004262932576239109, batch: 700
Loss: 0.04197099432349205, batch: 800
Loss: 0.10210076719522476, batch: 900
Test 2 Error: 
 Accuracy: 98.9%, Avg loss: 0.067956 

Loss: 0.00013056941679678857, batch: 0
Loss: 0.001052644569426775, batch: 100
Loss: 0.002975482027977705, batch: